# Initialize libraries

In [ ]:
import deeplabcut
from datetime import datetime

from dlchelperclass import DlcHelperClass as dhc

# Create new project

In [ ]:
# Enter your project name
project_name = 'hrnet_w48_2'
# If you want to use labeled frames from an earlier project,
# make sure to use the same experiment name as in the previous project.
# Enter your experimenter name
experimenter_name = 'conv_vid'
# Specify where you want your project folder to be created
work_dir = '/local/data2/LIA_LIU_PONTUS/LIA_LIU'
# create project path
project_path = work_dir + '/' + project_name + '-' + experimenter_name + '-' + datetime.now().strftime("%Y-%m-%d")
# video path
videos_dir_path = '/local/data2/LIA_LIU/videos_converted'
# Creates a list variable of all videos paths 
video_path = dhc.get_video_paths(path=videos_dir_path)
# Create project
deeplabcut.create_new_project(
    project=project_name,
    experimenter=experimenter_name,
    videos=video_path,
    working_directory=work_dir,
    copy_videos=True
)

# Config yaml path
config_path = project_path + '/config.yaml'

# Initialize paths (For loading existing project)
# skip this cell if you created a new project

In [ ]:
# Enter your project path
project_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/dlcrnet_stride32_ms5_2-conv_vid-2024-11-18"
# Creates path for config file and videos
config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

# Extract frames

In [ ]:
# Extract frames for labeling, amount of images and train/test ratio defined in config.yaml
deeplabcut.extract_frames(config_path=config_path, video_path=video_path, mode='automatic')

# Use napari for labeling 

# Creates CSV and h5 files needed for labeling

In [ ]:
deeplabcut.create_training_dataset(config=config_path)

# Train network

In [ ]:
deeplabcut.train_network(config_path, maxiters=1_000_000)

# Creates training_stats.csv

In [ ]:
# Needed for plotting losses
deeplabcut.evaluate_network(config_path, plotting=True)

# Makes prediction on videos

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

# Create new videos with the models predicted labels

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# Evaluate models

In [ ]:
# Creates likelihood.txt file and loss/vall_loss plot
dhc.save_mean_likelihood_to_file(project_path=project_path)
dhc.plot_loss_to_png(project_path)

# Extract outlier frames and re-labeling

In [ ]:
deeplabcut.extract_outlier_frames(config=config_path,
                                   shuffle=1,
                                  automatic=True,
                                  videos=video_path)

In [ ]:
# Open the DeepLabCut GUI to manually correct labels
# Launch the GUI to manually adjust labels on the extracted frames
deeplabcut.refine_labels(project_path)

In [ ]:
deeplabcut.merge_datasets(config=config_path)

In [ ]:
# Create a new training dataset that includes the updated labels
# This prepares the training files with the corrected labels for training
deeplabcut.create_training_dataset(config=config_path)

# Finetune 

In [ ]:
# Make sure you have run the following code line before:
# * deeplabcut.analyze_videos(project_path, video_paths)

In [ ]:
# Retrain the model with the updated dataset
# Use transfer learning to improve the existing model with the new data
deeplabcut.train_network(project_path, shuffle=1, displayiters=100, saveiters=1000)

In [ ]:
# Re-evaluate performance on the updated model
# Analyze the videos again to check if model accuracy has improved after retraining
deeplabcut.analyze_videos(project_path, video_path)

# Predict unseen data

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)